In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression


In [2]:
import pymysql

# MariaDB 연결 설정
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='',
    db='charging',
    charset='utf8'
)


In [3]:
results = []

# SQL 쿼리 실행
with conn.cursor() as cursor:
    sql_queries = [
        "select * from charging.signet_payment", 
        "select * from charging.stcc_payment", 
        "select * from charging.open_info"
    ]
    
    for sql in sql_queries:
        cursor.execute(sql)
        result = cursor.fetchall()
        results.append(result)


In [4]:
signet_payment_df, stcc_payment_df, open_info_df = results


In [5]:
# 예시 코드
column_names = []

# SQL 쿼리 실행
with conn.cursor() as cursor:
    sql_queries = [
        "select * from charging.signet_payment", 
        "select * from charging.stcc_payment", 
        "select * from charging.open_info"
    ]
    
    for sql in sql_queries:
        cursor.execute(sql)
        # 각 쿼리의 컬럼 이름 가져오기
        columns = [desc[0] for desc in cursor.description]
        column_names.append(columns)

column_names


[['platform',
  'area_1',
  'area_2',
  'station_id',
  'station_nm',
  'address',
  'charger_id',
  'charger_type',
  'charge_start',
  'charge_end',
  'charge_time',
  'm_id',
  'user_nm',
  'card_no',
  'charge_amt',
  'rate',
  'payment_amt',
  'update_dt'],
 ['platform',
  'charger_type',
  'charge_idx',
  'meter_raw',
  'user_nm',
  'charge_start',
  'charge_end',
  'charge_amt_kwh',
  'power_amt_kw',
  'rate',
  'payment_amt',
  'charger_status',
  'charge_cnt',
  'meter_no',
  'charger_no',
  'qrcode_no',
  'update_dt'],
 ['platform',
  'station_id',
  'station_nm',
  'charger_id',
  'charger_type',
  '22_oct',
  '22_nov',
  '22_dec',
  '23_jan',
  '23_feb',
  '23_mar',
  '23_apr',
  '23_may',
  '23_jun',
  '23_jul',
  '23_aug',
  'open_dt',
  'remark',
  'kw',
  'install_dt',
  'contract_1',
  'contract_2',
  'contract',
  'mixed',
  'address',
  'area_1',
  'area_2',
  'as_div',
  'as_team',
  'sales_div',
  'sales_team',
  'install_rate',
  'facility_type_2',
  'add_dupl',
 

In [9]:
signet_payment_df = pd.DataFrame(results[0], columns=column_names[0])
stcc_payment_df = pd.DataFrame(results[1], columns=column_names[1])
open_info_df = pd.DataFrame(results[2], columns=column_names[2])


In [10]:
# `charge_start` 컬럼을 datetime 형태로 변환
signet_payment_df['charge_start'] = pd.to_datetime(signet_payment_df['charge_start'])
stcc_payment_df['charge_start'] = pd.to_datetime(stcc_payment_df['charge_start'], errors='coerce')

# 두 데이터 프레임을 병합
combined_data = pd.concat([signet_payment_df, stcc_payment_df], ignore_index=True)

# 월별로 충전량을 집계
combined_data['year_month'] = combined_data['charge_start'].dt.to_period('M')
monthly_charge = combined_data.groupby(['station_id', 'year_month'])['charge_amt'].sum().reset_index()

# 연도와 월 특성 추가
monthly_charge['year'] = monthly_charge['year_month'].dt.year
monthly_charge['month'] = monthly_charge['year_month'].dt.month

# open_info 데이터와 병합하여 충전소의 특성 추가
monthly_charge_with_features = pd.merge(monthly_charge, open_info_df, left_on='station_id', right_on='station_id', how='left')

# 불필요한 컬럼 제거
monthly_charge_with_features.drop(['year_month'], axis=1, inplace=True)


In [11]:
from sklearn.model_selection import train_test_split


In [12]:
# 범주형 특성을 원-핫 인코딩
categorical_columns = ['platform', 'area_1', 'area_2', 'charger_type']
encoded_data = pd.get_dummies(monthly_charge_with_features, columns=categorical_columns)

# 예측 변수와 목표 변수 분할
X = encoded_data.drop('charge_amt', axis=1)
y = encoded_data['charge_amt']

# 데이터를 훈련 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:
# X_train 데이터프레임 내의 문자열 컬럼 확인
string_columns = X_train.select_dtypes(include=['object']).columns
string_columns


Index(['station_id', 'station_nm', 'charger_id', '22_oct', '22_nov', '22_dec',
       '23_jan', '23_feb', '23_mar', '23_apr', '23_may', '23_jun', '23_jul',
       '23_aug', 'open_dt', 'remark', 'kw', 'install_dt', 'contract_1',
       'contract_2', 'contract', 'mixed', 'address', 'as_div', 'as_team',
       'sales_div', 'sales_team', 'install_rate', 'facility_type_2',
       'add_dupl', 'add_dupl_id', 'update_dt', 'haus_no', 'parking_cnt'],
      dtype='object')

In [14]:
selected_columns = ['station_id', 'station_nm', 'charger_id', '23_aug', 'open_dt', 'remark',
       'kw', 'install_dt', 'contract_1', 'contract_2', 'contract', 'mixed',
       'address', 'as_div', 'as_team', 'sales_div', 'sales_team',
       'install_rate', 'facility_type_2', 'add_dupl', 'add_dupl_id',
       'update_dt', 'haus_no', 'parking_cnt']

X_train = X_train[selected_columns]
X_test = X_test[selected_columns]


In [15]:
X_train_encoded = pd.get_dummies(X_train, columns=['station_nm', 'charger_id', 'remark', 'address', 'as_div', 'as_team', 'sales_div', 'sales_team', 'facility_type_2'])
X_test_encoded = pd.get_dummies(X_test, columns=['station_nm', 'charger_id', 'remark', 'address', 'as_div', 'as_team', 'sales_div', 'sales_team', 'facility_type_2'])


In [16]:
# 원-핫 인코딩할 카테고리컬 컬럼 선택
categorical_columns = ['station_id', 'station_nm', 'charger_id', 'remark', 'address', 'as_div', 'as_team', 'sales_div', 'sales_team', 'facility_type_2']

# 선택한 컬럼들을 원-핫 인코딩
X_train_encoded = pd.get_dummies(X_train, columns=categorical_columns)
X_test_encoded = pd.get_dummies(X_test, columns=categorical_columns)


In [ ]:
# X_train_encoded와 X_test_encoded에 원-핫 인코딩 적용 후에 문자열 값을 숫자로 변환하는 예시
X_train_encoded = X_train_encoded.apply(lambda col: pd.to_numeric(col, errors='coerce'))
X_test_encoded = X_test_encoded.apply(lambda col: pd.to_numeric(col, errors='coerce'))

In [ ]:
# NaN 값은 적절한 값으로 대체
X_train_encoded.fillna(0, inplace=True)
X_test_encoded.fillna(0, inplace=True)


In [ ]:
# 선형 회귀 모델 훈련
lr_model = LinearRegression()
lr_model.fit(X_train_encoded, y_train)

In [ ]:
# 테스트 세트에서의 예측값 계산
test_predictions = lr_model.predict(X_test_encoded)

In [19]:
# X_train_encoded와 X_test_encoded에 원-핫 인코딩 적용 후에 문자열 값을 숫자로 변환하는 예시
X_train_encoded = X_train_encoded.apply(lambda col: pd.to_numeric(col, errors='coerce'))
X_test_encoded = X_test_encoded.apply(lambda col: pd.to_numeric(col, errors='coerce'))

# NaN 값은 적절한 값으로 대체
X_train_encoded.fillna(0, inplace=True)
X_test_encoded.fillna(0, inplace=True)

# 선형 회귀 모델 훈련
lr_model = LinearRegression()
lr_model.fit(X_train_encoded, y_train)

# 테스트 세트에서의 예측값 계산
test_predictions = lr_model.predict(X_test_encoded)
